In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score,train_test_split,GridSearchCV,KFold

In [4]:
X_test_raw = pd.read_csv("X_test_new_2.csv", index_col=0).values
X_train_raw = pd.read_csv("X_train_new_2.csv", index_col=0).values
y_train_raw = pd.read_csv("y_train.csv", index_col=0).values

In [5]:
from sklearn.preprocessing import StandardScaler,RobustScaler,PowerTransformer
def standardize_data(X_train,X_test) :
    r_train = RobustScaler()
    r_test = RobustScaler()
    
    p_train = PowerTransformer()
    r_train.fit(X_train)
    r_test.fit(X_test)
    
    scaled_train_data = r_train.transform(X_train)
    scaled_test_data = r_train.transform(X_test)
    
    return scaled_train_data,scaled_test_data

In [6]:
X_train_1,X_test_1 = standardize_data(X_train_raw,X_test_raw)

In [10]:
clf = RandomForestClassifier(n_estimators=100,
                             criterion='gini', 
                             max_depth=None, 
                             min_samples_split=2,
                             min_samples_leaf=1, 
                             min_weight_fraction_leaf=0.0,
                             max_features='sqrt', 
                             max_leaf_nodes=None,
                             min_impurity_decrease=0.0, 
                             bootstrap=True, 
                             oob_score=False,
                             n_jobs=None, 
                             random_state=None, 
                             verbose=0, 
                             warm_start=False,
                             class_weight=None, 
                             ccp_alpha=0.0, 
                             max_samples=None)
clf.fit(X_train_1,y_train_raw.ravel())
pred = clf.predict(X_test_1)

In [11]:
submission = np.hstack([np.arange(0, len(pred)).reshape(-1,1), pred.reshape(-1,1)]) 
submission_pd = pd.DataFrame(submission, columns=['id','y'])
submission_pd.to_csv('submission.csv', index=None)

In [93]:
from sklearn.ensemble import IsolationForest
def outlier_detection(X_train, y_train):
    clf = IsolationForest(max_samples=100, random_state = 4)
    preds = clf.fit_predict(X_train)
    X_train_1 = X_train[preds==1]
    y_train_1 = y_train[preds==1]
    return X_train_1, y_train_1

X_train_2,y_train_2= outlier_detection(X_train_1, y_train_raw)

In [92]:
X_train_1.shape

(5117, 493)

In [94]:
X_train_2.shape

(4552, 493)

In [7]:
pd.DataFrame(X_train_1).head(20)

,0,1,2,3,4,5,6,7,8,9,...,483,484,485,486,487,488,489,490,491,492
0,-0.437283,11.324702,0.119035,-0.827586,-0.423913,-0.063233,-0.030303,0.000944,-0.634277,-0.592320,...,0.15,0.678571,0.061820,-0.259259,0.902666,0.300817,1.741074,0.038550,-0.012644,0.893020
1,0.165243,0.380454,0.166345,0.120690,-0.413043,-0.201605,-0.196970,-0.198667,-0.246757,-0.207562,...,-0.95,-0.250000,0.033219,-0.074074,0.615675,0.295694,0.007824,0.138730,0.158757,0.023448
2,-0.138774,-0.281788,-0.267372,0.491379,0.021739,0.331535,0.318182,-0.259823,-0.389069,-0.292726,...,0.55,-0.428571,0.275916,0.444444,-0.416429,-0.218770,-0.418872,-0.371190,-0.303386,-0.199231
3,0.902878,0.702110,3.895225,-0.801724,-0.989130,-1.957763,-2.060606,0.051827,2.314668,2.302418,...,1.35,0.714286,1.037616,1.000000,-0.333900,3.211963,2.412509,3.955306,4.169896,1.088846
4,0.531289,0.199302,1.477044,-1.103448,-0.804348,-1.073215,-1.075758,0.096700,-0.596560,-0.785713,...,-0.35,-0.500000,0.421104,0.666667,-0.134101,1.079097,0.658543,1.323777,1.452337,0.297830
5,-1.099861,-0.686025,-0.788633,-0.189655,3.228261,0.993055,0.886364,2.266433,0.258509,0.242827,...,-0.40,-1.000000,-0.601387,-0.259259,-0.305168,-1.045606,-0.160233,-0.429439,-0.312671,0.662753
6,-0.197197,0.870297,0.606443,0.008621,-0.706522,-0.534772,-0.545455,-0.300713,-0.400708,-0.415200,...,0.10,0.035714,0.688276,1.222222,0.496591,0.820696,0.185862,0.531539,0.602661,-0.064510
7,0.591597,0.287134,1.105650,-0.620690,0.978261,-0.842992,-0.992424,1.043028,0.363843,0.286049,...,-0.20,0.428571,0.801445,1.074074,0.572222,-0.397379,1.411587,1.301945,1.362036,1.576827
8,-0.905857,12.291612,-0.212686,-0.456897,1.728261,0.287679,0.333333,1.714874,-0.013373,-0.069576,...,0.05,0.250000,-0.239253,-0.259259,0.136953,-0.980137,0.605938,0.053974,0.210664,1.122311
9,0.314776,-14.684118,-1.049204,1.137931,0.771739,1.499147,1.409091,-0.245192,-0.151860,0.444670,...,0.40,-1.035714,-0.146556,0.148148,-0.561316,-0.916312,-0.906998,-1.126371,-1.054905,-0.393775


In [57]:
clf = XGBClassifier(max_depth = 11,
                    n_estimators = 100,#110
                    learning_rate = 0.07,#0.09,
                    subsample = 0.8,#0.8,
                    colsample_bytree = 0.9,#0.8,
                    min_child_weight = 2.0,
                    gamma = 0.9,#0.6,
                    # reg_alpha = reg_alpha,
                    # reg_lambda = reg_lambda,
                    # scale_pos_weight = 1,
                    # objective = 'multi:softmax',
                    # num_class = 4,
                    eval_metric = 'auc',
                    use_label_encoder=False
)

#ret = cross_val_score(clf, X_train, y_train_raw, scoring='f1_micro', cv=5)
#print(ret)

#clf.fit(X_train_2, y_train_raw)
#print(np.sort(clf.feature_importances_))
#sf = np.argsort(clf.feature_importances_)[::-1][:200]
#print(sf)

strKFold = StratifiedKFold(n_splits=5,shuffle=False)

ret = cross_val_score(clf, X_train_1, y_train_raw, scoring='f1_micro', cv=strKFold)
print(ret)
print(np.mean(ret))

clf.fit(X_train_1, y_train_raw)
pred = clf.predict(X_test_1)
#clf.fit(X_train_raw[:,sf], y_train_raw)
#pred = clf.predict(X_test_raw[:,sf])

submission = np.hstack([np.arange(0, len(pred)).reshape(-1,1), pred.reshape(-1,1)]) 
submission_pd = pd.DataFrame(submission, columns=['id','y'])
submission_pd.to_csv('submission.csv', index=None)

[0.80566406 0.82324219 0.81915934 0.81622678 0.81427175]
0.8157128238025416


NameError: name 'y_train_1' is not defined

In [97]:
X_train_2,y_train_2= outlier_detection(X_train_1[:,sf], y_train_raw)

In [96]:
clf.fit(X_train_1, y_train_raw)
print(np.sort(clf.feature_importances_))
sf = np.argsort(clf.feature_importances_)[::-1][:400]
print(sf)

[0.         0.         0.         0.         0.         0.00101945
 0.00112462 0.00112964 0.00113048 0.00115756 0.00120441 0.00121844
 0.00123708 0.0012458  0.00125423 0.00125452 0.00125566 0.0012578
 0.00126913 0.0012692  0.00127017 0.00127034 0.00127598 0.0012801
 0.00128196 0.00128871 0.00128906 0.00129013 0.00129222 0.0012948
 0.00130747 0.00131837 0.00132142 0.00132397 0.00133008 0.00133671
 0.00134253 0.0013448  0.00134944 0.00135243 0.00135424 0.00135498
 0.00135687 0.00135744 0.0013606  0.00136766 0.00137457 0.00138504
 0.00139479 0.00139494 0.00140337 0.00140709 0.00140888 0.00140942
 0.00141347 0.00141468 0.00141567 0.00141575 0.00141618 0.00142571
 0.00142644 0.00143093 0.00143212 0.00143229 0.00143266 0.00144509
 0.00144698 0.0014479  0.00144803 0.00144999 0.00145313 0.00145379
 0.00145478 0.00145496 0.00145952 0.00145961 0.0014607  0.00146558
 0.00146598 0.00146661 0.00146879 0.00147121 0.00147474 0.00147562
 0.00147712 0.0014815  0.00148294 0.00148581 0.00148603 0.0014900

In [101]:
clf.fit(X_train_2, y_train_2)
pred = clf.predict(X_test_1[:,sf])

In [100]:
submission = np.hstack([np.arange(0, len(pred)).reshape(-1,1), pred.reshape(-1,1)]) 
submission_pd = pd.DataFrame(submission, columns=['id','y'])
submission_pd.to_csv('submission.csv', index=None)

In [99]:
pd.DataFrame(pred).head(20)

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,1
7,0
8,2
9,0


In [59]:
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp,space_eval,rand,Trials,partial,STATUS_OK
results = []

def GBM(argsDict):
    max_depth = argsDict["max_depth"] + 7
    n_estimators = argsDict['n_estimators'] * 5 + 90
    learning_rate = argsDict["learning_rate"] * 0.02 + 0.05
    subsample = argsDict["subsample"] * 0.1 + 0.5
    colsample_bytree = argsDict["colsample_bytree"] * 0.1 + 0.7
    min_child_weight = argsDict["min_child_weight"] * 0.5
    gamma = argsDict["gamma"] * 0.1 + 0.5
    # reg_alpha = argsDict["reg_alpha"] * 0.1
    # reg_lambda = argsDict["reg_lambda"] * 0.1
    print("max_depth:" + str(max_depth))
    print("n_estimators:" + str(n_estimators))
    print("learning_rate:" + str(learning_rate))
    print("subsample:" + str(subsample))
    print("colsample_bytree:" + str(colsample_bytree))
    print("min_child_weight:" + str(min_child_weight))
    print("gamma:" + str(gamma))
    # print("reg_alpha:" + str(reg_alpha))
    # print("reg_lambda:" + str(reg_lambda))

    global X_train_1, y_train_1#, X_test

    gbm = XGBClassifier(max_depth = max_depth,
                        n_estimators = n_estimators,#110
                        learning_rate = learning_rate,#0.09,
                        subsample = subsample,#0.8,
                        colsample_bytree = colsample_bytree,#0.8,
                        min_child_weight = min_child_weight,
                        gamma = gamma,#0.6,
                        # reg_alpha = reg_alpha,
                        # reg_lambda = reg_lambda,
                        # scale_pos_weight = 1,
                        # objective = 'multi:softmax',
                        # num_class = 4,
                        eval_metric = 'auc',
                        use_label_encoder=False
    )

    strKFold = StratifiedKFold(n_splits=5,shuffle=False)
    cv_results = cross_val_score(gbm,X_train_1,y_train_raw,cv=strKFold,scoring="f1_micro") #X_train_1, y_train_1 
    metric = cv_results.mean()
    results.append(metric)
    print(cv_results)
    print(str(metric) + '\n')
    return -metric

space = {
    "max_depth":hp.randint("max_depth",5), # + 7
    "n_estimators":hp.randint("n_estimators",5),  # * 5 + 75
    "learning_rate":hp.randint("learning_rate",6),  #[0,1,2,3,4,5] -> 0.05,0.06
    "subsample":hp.randint("subsample",5),#[0,1,2,3] -> [0.7,0.8,0.9]
    "colsample_bytree":hp.randint("colsample_bytree",3),#[0,1,2,3,4] -> [0.7,0.8,0.9]
    "min_child_weight":hp.randint("min_child_weight",10), #[0,1,2,3,4,5,6,7] -> +1
    "gamma":hp.randint("gamma", 5), # * 0.1 + 0.5
    # "reg_alpha":hp.randint("reg_alpha", 30), # * 0.1
    # "reg_lambda":hp.randint("reg_lambda", 30), # * 0.1
    }
algo = partial(tpe.suggest,n_startup_jobs=1)
best = fmin(GBM,space,algo=algo,max_evals=100)#max_evals

print(best)
print(GBM(best))

max_depth:7                                                                     
n_estimators:95                                                                 
learning_rate:0.11                                                              
subsample:0.5                                                                   
colsample_bytree:0.8999999999999999                                             
min_child_weight:2.0                                                            
gamma:0.9                                                                       
[0.81152344 0.81835938 0.82111437 0.81036168 0.81427175]                        
0.8151261226173021                                                              

max_depth:7                                                                     
n_estimators:95                                                                 
learning_rate:0.11                                                              
subsample:0.5              

KeyboardInterrupt: 